<center> <h1>Using GraphWave </h1> </center>

&nbsp;

&nbsp;

The goal of the  following notebook is to show how the GraphWave algorithm can be used. 

GraphWave was implemented in Python 2.7 and requires to load the following Python packages:

+ __pygsp__ (Graph signal Processing package from EPFL, to compute spectral graph wavelets.)
+ __networkx__ (for handling network objects: in particular, visualization, etc.)
+ traditional libraries for data analytics: __seaborn__ for plotting, __pandas__ for dataframes


In [24]:
%matplotlib inline
import networkx as nx 
import numpy as np
import pandas as pd
import seaborn as sb
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
from graphwave.shapes import build_graph
from graphwave.graphwave import *


np.random.seed(123)


## I. Creating a graph

In [25]:
# 1- Start by defining our favorite regular structure

filename_graph_indic="/Users/cdonnat/Downloads/Wiki-Vote.txt"
edge_list=[]
with open(filename_graph_indic) as f:
    i = 0
    for line in f:
        line=line.strip("\n")
        line=line.strip("\r")
        if i > 5:
            line = line.split('\t')
            edge_list += [[int(line[0]),int(line[1])]]
        i += 1

In [26]:
graph = nx.from_edgelist(edge_list)

In [27]:
graph.number_of_edges()

100760

In [5]:
help(nx.number_connected_components)

Help on function number_connected_components in module networkx.algorithms.components.connected:

number_connected_components(G)
    Return the number of connected components.
    
    Parameters
    ----------
    G : NetworkX graph
       An undirected graph.
    
    Returns
    -------
    n : integer
       Number of connected components
    
    See Also
    --------
    connected_components
    number_weakly_connected_components
    number_strongly_connected_components
    
    Notes
    -----
    For undirected graphs only.



In [6]:
nx.number_connected_components(graph)

24

In [7]:
conn_comp={}
it_g = 0
for g in nx.connected_component_subgraphs(graph):
    conn_comp[it_g] = g
    print(it_g, g.number_of_nodes())
    it_g += 1

(0, 7066)
(1, 2)
(2, 2)
(3, 2)
(4, 2)
(5, 2)
(6, 2)
(7, 2)
(8, 2)
(9, 2)
(10, 2)
(11, 2)
(12, 2)
(13, 2)
(14, 2)
(15, 3)
(16, 2)
(17, 2)
(18, 3)
(19, 2)
(20, 2)
(21, 2)
(22, 2)
(23, 3)


In [8]:
taus = list(np.arange(0.5, 3.0, 0.2))+list(range(3, 5))
# Compute the optimal embedding
pygsp_graph = pygsp.graphs.Graph(nx.adjacency_matrix(conn_comp[0]),
                                 lap_type='normalized')

In [9]:
pygsp_graph.L

<7066x7066 sparse matrix of type '<type 'numpy.float64'>'
	with 208534 stored elements in Compressed Sparse Row format>

In [10]:
pygsp_graph.connected


True

In [21]:
#pygsp_graph.compute_fourier_basis(recompute=True)
# safety check to ensure that the graph is indeed connected
l1 = sc.sparse.linalg.eigsh(pygsp_graph.L, k=2, which='SM')[0][1]
smax = -np.log(0.75) * np.sqrt(2.0 / l1)
smin = -np.log(0.90) * np.sqrt(2.0/ l1)
taus = list(np.linspace(smin,smax,10))

In [ ]:
import time
tic=time.time()
diff_type = 'heat'
n_nodes = pygsp_graph.N
if True:
    n_filters = len(taus)

    if diff_type == 'mexican':
        n_filters = 6
        Hk = pygsp.filters.MexicanHat(pygsp_graph, n_filters)
    elif diff_type == 'wave':
        Hk = pygsp.filters.Wave(pygsp_graph, taus, normalize=True)
    else:
        Hk = pygsp.filters.Heat(pygsp_graph, taus, normalize=False)

heat = {tau: sc.sparse.csc_matrix(( n_nodes, n_nodes)) for tau in taus}
Sf_vec = Hk.analysis(np.eye(n_nodes), cheb_order = 10)
toc =time.time()
print(toc-tic)

In [14]:
#### Then we need to convert them into an adequate format

In [13]:
help(Hk)

Help on Heat in module pygsp.filters.heat object:

class Heat(pygsp.filters.filter.Filter)
 |  Heat Filterbank
 |  
 |  Inherits its methods from Filters
 |  
 |  Parameters
 |  ----------
 |  G : Graph
 |  tau : int or list of ints
 |      Scaling parameter. (default = 10)
 |  normalize : bool
 |      Normalize the kernel (works only if the eigenvalues are
 |      present in the graph). (default = 0)
 |  
 |  Returns
 |  -------
 |  out : Heat
 |  
 |  Examples
 |  --------
 |  >>> from pygsp import graphs, filters
 |  >>> G = graphs.Logo()
 |  >>> F = filters.Heat(G)
 |  
 |  Method resolution order:
 |      Heat
 |      pygsp.filters.filter.Filter
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, G, tau=10, normalize=False, **kwargs)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from pygsp.filters.filter.Filter:
 |  
 |  analysis(self, s, method=None, cheb_order=30, lanczos_order=30, **kwargs)
 |

In [15]:
for i in range(n_filters):
        heat[taus[i]]= sc.sparse.csc_matrix(Sf_vec[i * n_nodes: (i + 1) * n_nodes, :] )    # stores in tensor the results

In [20]:
np.sort(heat[taus[9]][:,0].todense())

matrix([[  7.83515091e-01],
        [  7.34737288e-05],
        [  5.90369948e-05],
        ..., 
        [  1.19696857e-07],
        [  7.96622698e-09],
        [  2.41718737e-08]])

In [17]:
7066 * 7066

49928356

(Note: best visualization of the graphs are obtained using Gephi, or some other specialized graph visualization software)

## II. Running GraphWave


We propose here a simple demonstration of GraphWave using both the automatic version (part a) and the manual version. This shows how to use GraphWave in a parameter-free version, or giving the analyst the possibility to select an adequate scale value.

For each of these approaches, we compute the signature by calling GraphWave. We then compute its PCA projection to visualize the embeddings. Note that in this very simple examples, GraphWave recovers structura equivalence, as shown by the overlapping embeddings on the first principal components.

#### a. Multiscale GraphWave: Automatic selection of the range of scales

In [ ]:
chi, heat_print, taus = graphwave(graph, taus='automatic_large_matrix', verbose=False)

We now visualize the resulting embeddings by computing their PCA projections. We also run KMeans to assess how well the signatures that we have here generated enable the recovery of structural roles.

In [ ]:
nb_clust = len(np.unique(role_id))
pca = PCA(n_components=5)
trans_data = pca.fit_transform(StandardScaler().fit_transform(chi))
km = KMeans(n_clusters=nb_clust)
km.fit(trans_data)
labels_pred=km.labels_

######## Params for plotting
cmapx=plt.get_cmap('rainbow')
x=np.linspace(0,1,nb_clust+1)
col=[cmapx(xx) for xx in x ]
markers = {0:'*',1: '.', 2:',',3: 'o',4: 'v',5: '^',6: '<',7: '>',8: 3 ,9:'d',10: '+',11:'x',12:'D',13: '|',14: '_',15:4,16:0,17:1,18:2,19:6,20:7}

for c in np.unique(role_id):
    indc = [i for i,x in enumerate(role_id) if x==c]
    plt.scatter(trans_data[indc,0], trans_data[indc,1],
                c=np.array(col)[list(np.array(labels_pred)[indc])],
                marker=markers[c%len(markers)], s=300)

labels = role_id
for label,c, x, y in zip(labels,labels_pred, trans_data[:, 0], trans_data[:, 1]):
            plt.annotate(label,xy=(x, y), xytext=(0, 0), textcoords='offset points')
            

#### Uniscale GraphWave: Hand-selected value for tau

In [ ]:
### Select a scale of interest (here we select a particular range of scale. See associated paper for 
### guidelines on how to select the appropriate scale.)

### Compute the heat wavelet
from graphwave.graphwave import *

time_pts = list(np.arange(0,50,0.5))
chi,heat_print, taus=graphwave(G, taus = [0.5], time_pts=time_pts, verbose=False)
print(chi.shape, len(time_pts))

Note that in the EPFL implementation, by construction, the wavelet scales are all divided by the maximum eigenvalue $\lambda_N$.

In [ ]:
nb_clust=len(np.unique(role_id))
pca=PCA(n_components=5)
trans_data=pca.fit_transform(StandardScaler().fit_transform(chi))
km=KMeans(n_clusters=nb_clust)
km.fit(trans_data)
labels_pred=km.labels_


cmapx=plt.get_cmap('rainbow')
x=np.linspace(0,1,np.max(labels_pred)+1)
col=[cmapx(xx) for xx in x ]
markers = {0:'*',1: '.', 2:',',3: 'o',4: 'v',5: '^',6: '<',7: '>',8: 3 ,9:'d',10: '+',11:'x',12:'D',13: '|',14: '_',15:4,16:0,17:1,18:2,19:6,20:7}


for c in np.unique(role_id):
        indc=[i for i,x in enumerate(role_id) if x==c]
        _ = plt.scatter(trans_data[indc,0], trans_data[indc,1],c=np.array(col)[list(np.array(labels_pred)[indc])] ,marker=markers[c%len(markers)],s=500)

labels = role_id
for label,c, x, y in zip(labels,labels_pred, trans_data[:, 0], trans_data[:, 1]):
            _ = plt.annotate(label,xy=(x, y), xytext=(0, 0), textcoords='offset points')
            

## III. Visualizing the Characteristic functions

We now propose to show how to visualize characteristic functions.


In [ ]:
mapping = {u: i for i,u in enumerate(np.unique(role_id))}
cmap=plt.get_cmap('gnuplot')
role_id_plot=[cmap(x) for x in np.linspace(0,1,len(np.unique(role_id)))]
plt.figure()
ind_x=range(chi[0].shape[0])[0::2]
ind_y=range(chi[0].shape[0])[1::2]
for i in np.random.choice(range(G.number_of_nodes()),10,replace=False):
    _ = plt.plot(chi[i,ind_x],chi[i,ind_y],label=str(i),color=role_id_plot[mapping[role_id[i]]])

_ = plt.legend(loc='center left',bbox_to_anchor=(1,0.5))
